# Tugas NLP Implementasi Arsitektur Transformer
Raden Aryo Bismo Nugroho | 22/494473/TK/54233

## Inisialisasi dan Hyperparameters

In [52]:
import numpy as np

In [53]:
# Hyperparameters
vocab_size = 1000
d_model = 512
n_heads = 8
n_layers = 6
d_ff = 2048
max_seq_len = 100

In [54]:
# Ensure d_model is divisible by n_heads
assert d_model % n_heads == 0
d_k = d_model // n_heads

## Token Embedding

In [55]:
def token_embedding(tokens, vocab_size, d_model):
    # Create the embedding matrix (lookup table)
    # In a real model, these weights would be learned. Here, we initialize them randomly.
    embedding_matrix = np.random.randn(vocab_size, d_model)

    # Retrieve the embeddings for the input tokens
    return embedding_matrix[tokens]

In [56]:
# Create a dummy input: batch of 2 sequences, each with 5 tokens
dummy_tokens = np.array([
    [10, 2, 45, 70, 33],
    [5, 88, 12, 21, 9]
])

embedding_output = token_embedding(dummy_tokens, vocab_size, d_model)
print("Shape of Token Embeddings:", embedding_output.shape)
# Expected output: (2, 5, 512)

Shape of Token Embeddings: (2, 5, 512)


## Positional Encoding (Sinusoidal)

In [57]:
def positional_encoding(seq_len, d_model):

    position = np.arange(seq_len)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))

    pe = np.zeros((seq_len, d_model))

    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)

    return pe

In [58]:
# Add the positional encoding to the token embeddings
seq_len = dummy_tokens.shape[1]
pos_encoding = positional_encoding(seq_len, d_model)
x = embedding_output + pos_encoding

print("Shape after adding Positional Encoding:", x.shape)
# Expected output: (2, 5, 512)

Shape after adding Positional Encoding: (2, 5, 512)


## Casual Masking

In [59]:
def create_causal_mask(seq_len):

    # np.triu returns the upper triangle of an array. k=1 offsets it by one.
    mask = np.triu(np.ones((seq_len, seq_len)), k=1)

    # set the masked values to a very small number (negative infinity), so they become zero after the softmax operation.
    return mask * -1e9

In [60]:
causal_mask = create_causal_mask(seq_len)
print("Causal Mask (for seq_len=5):")
print(causal_mask)
# Expected output: A 5x5 matrix with 0s on and below the diagonal, and -1e9 above it.

Causal Mask (for seq_len=5):
[[-0.e+00 -1.e+09 -1.e+09 -1.e+09 -1.e+09]
 [-0.e+00 -0.e+00 -1.e+09 -1.e+09 -1.e+09]
 [-0.e+00 -0.e+00 -0.e+00 -1.e+09 -1.e+09]
 [-0.e+00 -0.e+00 -0.e+00 -0.e+00 -1.e+09]
 [-0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00]]


## Scaled Dot-Product Attention

In [61]:
def softmax(x):
    # Computes softmax along the last axis, with numerical stability
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

In [62]:
def scaled_dot_product_attention(Q, K, V, mask=None):

    # Matrix multiplication Q and K.T
    # Q: (..., seq_len, d_k), K.T: (..., d_k, seq_len) -> scores: (..., seq_len, seq_len)
    scores = np.matmul(Q, K.swapaxes(-2, -1))

    # Scale the scores
    d_k = Q.shape[-1]
    scaled_scores = scores / np.sqrt(d_k)

    # Apply mask if provided
    if mask is not None:
        scaled_scores += mask

    # Get attention weights using softmax
    attention_weights = softmax(scaled_scores)

    # Matrix multiplication weights and V
    # weights: (..., seq_len, seq_len), V: (..., seq_len, d_k) -> output: (..., seq_len, d_k)
    output = np.matmul(attention_weights, V)

    return output, attention_weights

## Multi-Head Attention

In [63]:
class MultiHeadAttention:
    def __init__(self, d_model, n_heads):
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads

        # Initialize weights for Q, K, V projections and the final output projection
        self.W_q = np.random.randn(d_model, d_model)
        self.W_k = np.random.randn(d_model, d_model)
        self.W_v = np.random.randn(d_model, d_model)
        self.W_o = np.random.randn(d_model, d_model)

    def split_heads(self, x):
        # x shape: (batch_size, seq_len, d_model)
        batch_size, seq_len, _ = x.shape
        # Reshape to (batch_size, seq_len, n_heads, d_k)
        x = x.reshape(batch_size, seq_len, self.n_heads, self.d_k)
        # Transpose to (batch_size, n_heads, seq_len, d_k) for attention calculation
        return x.transpose(0, 2, 1, 3)

    def forward(self, x, mask):
        # x shape: (batch_size, seq_len, d_model)
        batch_size, seq_len, _ = x.shape

        # 1. Project to Q, K, V
        Q = x @ self.W_q
        K = x @ self.W_k
        V = x @ self.W_v

        # 2. Split into multiple heads
        Q = self.split_heads(Q)
        K = self.split_heads(K)
        V = self.split_heads(V)

        # 3. Apply scaled dot-product attention
        # Q, K, V shapes: (batch_size, n_heads, seq_len, d_k)
        attention_output, attention_weights = scaled_dot_product_attention(Q, K, V, mask)

        # print("Bukti Causal Masking (Attention Weights)")
        # print(attention_weights[0, 0, :, :])

        # 4. Concatenate heads
        # Transpose back to (batch_size, seq_len, n_heads, d_k)
        attention_output = attention_output.transpose(0, 2, 1, 3)
        # Reshape to (batch_size, seq_len, d_model)
        concatenated_output = attention_output.reshape(batch_size, seq_len, self.d_model)

        # 5. Final linear projection
        output = concatenated_output @ self.W_o
        return output

In [64]:
mha = MultiHeadAttention(d_model, n_heads)
# 'x' is our data after adding positional encoding
# We pass the same 'x' for Q, K, and V in self-attention
mha_output = mha.forward(x, causal_mask)

print("Shape of Multi-Head Attention output:", mha_output.shape)
# Expected output: (2, 5, 512)

Bukti Causal Masking (Attention Weights)
[[1.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [2.99041815e-058 1.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [5.50377254e-154 0.00000000e+000 1.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 1.00000000e+000
  0.00000000e+000]
 [6.52441429e-226 0.00000000e+000 0.00000000e+000 1.07800040e-214
  1.00000000e+000]]
Shape of Multi-Head Attention output: (2, 5, 512)


## Layer Normalization dan Feed-Forward Network

In [65]:
class LayerNormalization:
    def __init__(self, d_model, epsilon=1e-5):
        self.epsilon = epsilon
        # Gamma and Beta are learnable parameters
        self.gamma = np.ones(d_model)
        self.beta = np.zeros(d_model)

    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        mean = np.mean(x, axis=-1, keepdims=True)
        std = np.std(x, axis=-1, keepdims=True)

        # Normalize
        normalized_x = (x - mean) / (std + self.epsilon)

        # Scale and shift
        return self.gamma * normalized_x + self.beta

In [66]:
class FeedForwardNetwork:
    def __init__(self, d_model, d_ff):
        # Initialize weights and biases
        self.W1 = np.random.randn(d_model, d_ff)
        self.b1 = np.zeros(d_ff)
        self.W2 = np.random.randn(d_ff, d_model)
        self.b2 = np.zeros(d_model)

    def relu(self, x):
        return np.maximum(0, x)

    def forward(self, x):
        # First linear layer + ReLU
        hidden = self.relu((x @ self.W1) + self.b1)
        # Second linear layer
        output = (hidden @ self.W2) + self.b2
        return output

In [67]:
ffn = FeedForwardNetwork(d_model, d_ff)
ln1 = LayerNormalization(d_model)
ln2 = LayerNormalization(d_model)

## Pre-Norm & Residual Connection

In [68]:
# This is the "pre-norm" flow for one block
# First sublayer: Multi-Head Attention
norm_x = ln1.forward(x)
mha_output = mha.forward(norm_x, causal_mask)
# Residual connection
add_mha = x + mha_output

# Second sublayer: Feed-Forward Network
norm_add_mha = ln2.forward(add_mha)
ffn_output = ffn.forward(norm_add_mha)
# Residual connection
add_ffn = add_mha + ffn_output

print("Shape after one Transformer block:", add_ffn.shape)
# Expected output: (2, 5, 512)

Bukti Causal Masking (Attention Weights)
[[1.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [7.13835174e-167 1.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [4.29528361e-273 0.00000000e+000 1.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [3.41595332e-313 4.87664770e-165 8.96140256e-243 1.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  1.00000000e+000]]
Shape after one Transformer block: (2, 5, 512)


## The Complete Tranformer Model

In [69]:
class TransformerBlock:
    def __init__(self, d_model, n_heads, d_ff):
        self.attention = MultiHeadAttention(d_model, n_heads)
        self.ffn = FeedForwardNetwork(d_model, d_ff)
        self.ln1 = LayerNormalization(d_model)
        self.ln2 = LayerNormalization(d_model)

    def forward(self, x, mask):
        # Pre-norm attention sublayer
        norm_x = self.ln1.forward(x)
        attention_output = self.attention.forward(norm_x, mask)
        x = x + attention_output # Residual connection

        # Pre-norm feed-forward sublayer
        norm_x = self.ln2.forward(x)
        ffn_output = self.ffn.forward(norm_x)
        x = x + ffn_output # Residual connection

        return x

In [70]:
class DecoderOnlyTransformer:
    def __init__(self, vocab_size, d_model, n_layers, n_heads, d_ff, max_seq_len):
        self.embedding_matrix = np.random.randn(vocab_size, d_model)
        self.pos_encoding_matrix = positional_encoding(max_seq_len, d_model)

        self.blocks = [TransformerBlock(d_model, n_heads, d_ff) for _ in range(n_layers)]

        self.final_ln = LayerNormalization(d_model)

        # The final linear layer to project to vocab size
        self.output_projection = np.random.randn(d_model, vocab_size)

    def forward(self, tokens):
        batch_size, seq_len = tokens.shape

        # 1. Create mask
        mask = create_causal_mask(seq_len)

        # 2. Input embedding and positional encoding
        x = self.embedding_matrix[tokens]
        x += self.pos_encoding_matrix[:seq_len, :]

        # 3. Pass through Transformer blocks
        for block in self.blocks:
            x = block.forward(x, mask)

        # 4. Final normalization
        x = self.final_ln.forward(x)

        # 5. Project to vocabulary space
        # Logits are the raw scores before softmax
        logits = x @ self.output_projection

        # 6. Get probability distribution
        probs = softmax(logits)

        return logits, probs

In [71]:
# FInal Test

# Instantiate the full model
model = DecoderOnlyTransformer(vocab_size, d_model, n_layers, n_heads, d_ff, max_seq_len)

# Run the forward pass with our dummy tokens
final_logits, final_probs = model.forward(dummy_tokens)

print("\n--- Final Output ---")
print("Shape of Logits:", final_logits.shape)
# Expected output: (2, 5, 1000)
print("Shape of Probabilities:", final_probs.shape)
# Expected output: (2, 5, 1000)

# To predict the NEXT token, we only care about the last position in the sequence
next_token_probs = final_probs[:, -1, :]
print("\nProbabilities for the next token (shape):", next_token_probs.shape)
# Expected output: (2, 1000)

# Check if probabilities sum to 1 for each sequence in the batch
print("Sum of probabilities for next token (should be ~1.0):", np.sum(next_token_probs[0]))

Bukti Causal Masking (Attention Weights)
[[1.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [1.00000000e+000 9.54312342e-016 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 1.00000000e+000 7.52482974e-176 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 1.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 1.00000000e+000 4.98354934e-077 1.38915299e-172
  0.00000000e+000]]
Bukti Causal Masking (Attention Weights)
[[1.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [4.51447463e-017 1.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 1.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 1.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [3.38736114e-258 7.64835387e-193 1.00000000e+000 3.89727748e-125
  3.94291144e-217]]
Bukti Causal Masking (Attention Weights)
[[1.00000000e+000 0.00000